### IMPORTED REQUIRED LIBRARIES

In [1]:
import requests
from bs4 import BeautifulSoup

### Get Info Box for One Disney Movie and Store in Python Dictionary

### GET Request for URL to Extract Content

In [2]:
response = requests.get("https://en.wikipedia.org/wiki/Toy_Story_3")
print("Status Code =", response.status_code)

html_doc = response.content

# Created BeatufulSoup Object to Extract Information
soup = BeautifulSoup(html_doc, 'html.parser')
#print(soup.prettify())

Status Code = 200


In [3]:
table_box = soup.find(name="table",attrs={"class":"infobox vevent"})
table_row = table_box.find_all("tr")

# Below code will show the html code for movie info box
#for row in table_row:
#    print(row.prettify())

In [4]:
def get_value(row_data):
    lst = []
    if row_data.find("li"):
        for list_item in row_data.find_all("li"):
             lst.append(list_item.get_text(" ", strip=True).replace("\xa0"," "))
        return lst
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0"," ")

movie_info = {}

for index,row in enumerate(table_row):
    if index == 0:
        movie_info["Title"] = row.find("th").get_text(" ", strip=True).replace("\xa0"," ")
    elif index == 1:
        continue
    else:
        key = row.find("th").get_text(" ", strip=True).replace("\xa0"," ")
        value = get_value(row.find("td"))
        movie_info[key] = value
        
movie_info

{'Title': 'Toy Story 3',
 'Directed by': 'Lee Unkrich',
 'Screenplay by': 'Michael Arndt',
 'Story by': ['John Lasseter', 'Andrew Stanton', 'Lee Unkrich'],
 'Produced by': 'Darla K. Anderson',
 'Starring': ['Tom Hanks',
  'Tim Allen',
  'Joan Cusack',
  'Don Rickles',
  'Wallace Shawn',
  'John Ratzenberger',
  'Estelle Harris',
  'Ned Beatty',
  'Michael Keaton',
  'Jodi Benson',
  'John Morris'],
 'Cinematography': ['Jeremy Lasky', 'Kim White'],
 'Edited by': 'Ken Schretzmann',
 'Music by': 'Randy Newman',
 'Production companies': ['Walt Disney Pictures', 'Pixar Animation Studios'],
 'Distributed by': 'Walt Disney Studios Motion Pictures',
 'Release dates': ['June 12, 2010 ( 2010-06-12 ) ( Taormina Film Fest )',
  'June 18, 2010 ( 2010-06-18 ) (United States)'],
 'Running time': '103 minutes [1]',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$200 million [1]',
 'Box office': '$1.067 billion [1]'}

### Get Info Box for all Disney Movie

In [5]:
main_url = "https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films"
response = requests.get(main_url)
print("Status Code =", response.status_code)

html_doc = response.content

# Created BeatufulSoup Object to Extract Information
soup = BeautifulSoup(html_doc, 'html.parser')
#print(soup.prettify())

Status Code = 200


In [6]:
# Get all the movie link mention in main_url page
movies = soup.select(".wikitable.sortable i a")

# Showed first 5 movie link for checking
movies[0:5]

[<a href="/wiki/Snow_White_and_the_Seven_Dwarfs_(1937_film)" title="Snow White and the Seven Dwarfs (1937 film)">Snow White and the Seven Dwarfs</a>,
 <a href="/wiki/Pinocchio_(1940_film)" title="Pinocchio (1940 film)">Pinocchio</a>,
 <a href="/wiki/Fantasia_(1940_film)" title="Fantasia (1940 film)">Fantasia</a>,
 <a href="/wiki/The_Reluctant_Dragon_(1941_film)" title="The Reluctant Dragon (1941 film)">The Reluctant Dragon</a>,
 <a href="/wiki/Dumbo" title="Dumbo">Dumbo</a>]

In [7]:
len(movies)

570

In [8]:
def get_value(row_data):
    lst = []
    if row_data.find("li"):
        for list_item in row_data.find_all("li"):
             lst.append(list_item.get_text(" ", strip=True).replace("\xa0"," "))
        return lst
    if row_data.find("br"):
        return [text for text in row_data.stripped_strings]
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0"," ")

# This functin removes the subscript from movie_data and extra release date in span tag   
def clean_tags(soup):
    for tag in soup.find_all(["sup","span"]):
        tag.decompose()

def get_movie_data(url):

    response = requests.get(f'https://en.wikipedia.org/{url}')
    html_doc = response.content
    
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    table_box = soup.find(name="table",attrs={"class":"infobox vevent"})
    table_row = table_box.find_all("tr")
    
    clean_tags(table_box)
    
    movie_info = {}

    for index,row in enumerate(table_row):
        if index == 0:
            movie_info["Title"] = row.find("th").get_text(" ", strip=True).replace("\xa0"," ")
        else:
            header = row.find('th')
            body = row.find('td')
            if header and body:
                key = row.find("th").get_text(" ", strip=True).replace("\xa0"," ")
                value = get_value(row.find("td"))
                movie_info[key] = value

    return movie_info

In [9]:
get_movie_data("wiki/The_Secret_of_the_Magic_Gourd_(2007_film)")

{'Title': 'The Secret of the Magic Gourd',
 'Traditional Chinese': '',
 'Simplified Chinese': '',
 'Directed by': ['John Chu', 'Frankie Chung'],
 'Written by': 'Zhang Tianyi',
 'Screenplay by': ['John Chu', 'Lam Kee To'],
 'Produced by': ['Han San Ping', 'John Chu'],
 'Edited by': 'Angie Lam',
 'Music by': ['Zhang Hanyun - "One Dream"', '張含韻《一人一夢》(Simplified Chinese)'],
 'Production companies': ['Walt Disney Pictures China',
  'Centro Digital Pictures Limited'],
 'Distributed by': 'China Film Group',
 'Release date': ['June 29, 2007 (China)'],
 'Running time': '85 minutes',
 'Country': 'China',
 'Language': 'Chinese',
 'Box office': '3 130 137'}

In [10]:
for index,movie in enumerate(movies):
    if movie['title'] == "Young Woman and the Sea":
        print(index)
        
# Note - From "Young Woman and the Sea" we have upcoming movie tittle 
# Thst's why we limit the index upto 525

526


In [11]:
movie_info_list = []

# Iterate through each movie link and extract movie info using get_movie_data function
for index,movie in enumerate(movies[:525]):
    try:
        movie_title = movie['title']
        movie_link = movie['href']
        movie_info_list.append(get_movie_data(movie_link))
        
    except Exception as e:
        print(index)
        print(movie.get_text())
        print(e)

In [12]:
len(movie_info_list)

525

### Save and Load Movie Data

In [13]:
import json

def save_json_data(filename,data):
    with open(filename,'w',encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=2)
        
def load_json_data(filename):
    with open(filename,'r',encoding='utf-8') as file:
        return json.load(file)

In [14]:
save_json_data("disney_movie_data.json",movie_info_list)

### Add IMDB and Rotten Tomatoes Rating Data for the Movies via API Key

In [15]:
movie_info_list = load_json_data("disney_movie_data.json")

In [16]:
import requests


def get_omdb_info(title):
    OMDB_API_KEY = "38f108cd"
    base_url = "https://www.omdbapi.com/?"
    base_params = {
        "apikey": OMDB_API_KEY,
        "t": title,
    }
    response = requests.get(base_url, params=base_params)
    result = response.json()
    return result

def get_Rotten_tomatoes_rating(omdb_info):
    ratings = omdb_info.get('Ratings', [])
    for rating in ratings:
        if rating['Source'] == "Rotten Tomatoes":
            return rating['Value']
    return None
    
get_omdb_info("Cinderella")

{'Title': 'Cinderella',
 'Year': '2015',
 'Rated': 'PG',
 'Released': '13 Mar 2015',
 'Runtime': '105 min',
 'Genre': 'Adventure, Drama, Family',
 'Director': 'Kenneth Branagh',
 'Writer': 'Chris Weitz, Charles Perrault',
 'Actors': 'Lily James, Cate Blanchett, Richard Madden',
 'Plot': "When her father unexpectedly dies, young Ella finds herself at the mercy of her cruel stepmother and her scheming stepsisters. Never one to give up hope, Ella's fortunes begin to change after meeting a dashing stranger.",
 'Language': 'English',
 'Country': 'United Kingdom, United States, Denmark, Canada, Italy',
 'Awards': 'Nominated for 1 Oscar. 9 wins & 38 nominations total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMjMxODYyODEzN15BMl5BanBnXkFtZTgwMDk4OTU0MzE@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.9/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '84%'},
  {'Source': 'Metacritic', 'Value': '67/100'}],
 'Metascore': '67',
 'imdbRating': '6.9',
 'imdb

In [17]:
for movie in movie_info_list:
    title = movie['Title']
    omdb_info = get_omdb_info(title)
    movie['imdb_rating'] = omdb_info.get('imdbRating', None)
    movie['imdb_votes'] = omdb_info.get('imdbVotes', None)
    movie['metascore'] = omdb_info.get('Metascore', None)
    movie['rotten_tomatoes_rating'] = get_Rotten_tomatoes_rating(omdb_info)

In [18]:
movie_info_list[-1]

{'Title': 'Wish',
 'Directed by': ['Chris Buck', 'Fawn Veerasunthorn'],
 'Screenplay by': ['Jennifer Lee', 'Allison Moore'],
 'Story by': ['Jennifer Lee',
  'Chris Buck',
  'Fawn Veerasunthorn',
  'Allison Moore'],
 'Produced by': ['Peter Del Vecho', 'Juan Pablo Reyes Lancaster-Jones'],
 'Starring': ['Ariana DeBose',
  'Chris Pine',
  'Alan Tudyk',
  'Angelique Cabral',
  'Victor Garber',
  'Natasha Rothwell',
  'Jennifer Kumiyama',
  'Harvey Guillén',
  'Evan Peters',
  'Ramy Youssef',
  'Jon Rudnitsky'],
 'Cinematography': ['Rob Dressel (layout)', 'Adolph Lusinsky (lighting)'],
 'Edited by': 'Jeff Draheim',
 'Music by': ['Dave Metzger (score)',
  'Julia Michaels (songs)',
  'Benjamin Rice (songs)'],
 'Production companies': ['Walt Disney Pictures',
  'Walt Disney Animation Studios'],
 'Distributed by': ['Walt Disney Studios', 'Motion Pictures'],
 'Release dates': ['November 8, 2023 ( El Capitan Theatre )',
  'November 22, 2023 (United States)'],
 'Running time': '95 minutes',
 'Count

### Save data as JSON

In [19]:
save_json_data("disney_movie_data.json",movie_info_list)

### Load Data In Pandas for Cleaning

In [20]:
import pandas as pd

In [21]:
df = pd.read_json("disney_movie_data.json")

In [22]:
df.shape

(525, 54)

In [23]:
pd.set_option("display.max_rows",None)
df.isnull().sum()

Title                       0
Directed by                 3
Story by                  358
Based on                  237
Produced by                11
Starring                   33
Music by                    4
Production company        303
Distributed by              3
Release dates             324
Running time                3
Country                    72
Language                   24
Budget                    209
Box office                123
imdb_rating                23
imdb_votes                 23
metascore                  23
rotten_tomatoes_rating    149
Narrated by               469
Cinematography            125
Release date              204
Written by                303
Edited by                  36
Languages                 504
Screenplay by             265
Countries                 460
Color process             518
Genre                     522
Created by                523
Country of origin         522
No. of seasons            523
No. of episodes           522
Executive 

In [24]:
pd.set_option("display.max_columns",None)
df[df['Release'].notnull()]

,Title,Directed by,Story by,Based on,Produced by,Starring,Music by,Production company,Distributed by,Release dates,Running time,Country,Language,Budget,Box office,imdb_rating,imdb_votes,metascore,rotten_tomatoes_rating,Narrated by,Cinematography,Release date,Written by,Edited by,Languages,Screenplay by,Countries,Color process,Genre,Created by,Country of origin,No. of seasons,No. of episodes,Executive producers,Network,Release,Production companies,Developed by,Voices of,Theme music composer,Opening theme,Ending theme,Composer,Original language,Producers,Editor,Kanji,Transcriptions,Revised Hepburn,Adaptation by,Traditional Chinese,Simplified Chinese,Original title,Layouts by
46,Zorro,NaN,NaN,NaN,NaN,"[Guy Williams, Gene Sheldon, Henry Calvin, Geo...",NaN,Walt Disney Productions,NaN,NaN,22–24 minutes,NaN,NaN,NaN,NaN,6.4,"4,151",N/A,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Action / Adventure, Comedy drama, Western, Su...",Johnston McCulley (original stories),United States,2.0,78 (+4 specials),"[Bill Anderson, Walt Disney]",ABC,"[October 10, 1957, –, July 2, 1959]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
221,Mighty Ducks: The Animated Series,NaN,NaN,"[The Mighty Ducks, by, Steven Brill]",NaN,NaN,NaN,NaN,NaN,NaN,22 minutes,NaN,NaN,NaN,NaN,6.4,"2,156",N/A,None,NaN,Lesley Hutchison,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Action / Adventure, Science fantasy, Superher...","[Marty Isenberg, Robert N. Skir, David Wise]",United States,1.0,26,NaN,"[ABC, Syndication]","[September 6, 1996, –, January 17, 1997]","[Walt Disney Television, Walt Disney Televisio...","[Marty Isenberg, Robert N. Skir, David Wise, G...","[Ian Ziering, James Belushi, Jeff Bennett, Cla...",Patrick DeRemer,"""Ducks Rock"" performed by Mickey Thomas","""Ducks Rock"" Instrumental",Carl Johnson,English,"[Blair Peters, Frank Squillace, Joe Barruso]",Jamie Thomason,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
500,The Beatles: Get Back,Peter Jackson,NaN,NaN,NaN,"[John Lennon, Paul McCartney, George Harrison,...",[The Beatles],NaN,NaN,NaN,"[157 minutes (part 1), 173 minutes (part 2), 1...",NaN,NaN,NaN,NaN,9.0,"27,340",N/A,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Music documentary,NaN,"[United Kingdom, New Zealand, United States]",NaN,3,"[Jeff Jones, Ken Kamins]",Disney+,"[25 November, –, 27 November 2021]","[Apple Corps Ltd., WingNut Films]",NaN,NaN,NaN,NaN,NaN,NaN,English,"[Paul McCartney, Ringo Starr, Yoko Ono Lennon,...",Jabez Olssen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df.isnull().sum()[df.isnull().sum()>400].index

Index(['Narrated by', 'Languages', 'Countries', 'Color process', 'Genre',
       'Created by', 'Country of origin', 'No. of seasons', 'No. of episodes',
       'Executive producers', 'Network', 'Release', 'Developed by',
       'Voices of', 'Theme music composer', 'Opening theme', 'Ending theme',
       'Composer', 'Original language', 'Producers', 'Editor', 'Kanji',
       'Transcriptions', 'Revised Hepburn', 'Adaptation by',
       'Traditional Chinese', 'Simplified Chinese', 'Original title',
       'Layouts by'],
      dtype='object')

In [26]:
df.drop(df.isnull().sum()[df.isnull().sum()>400].index,axis=1,inplace=True)

In [27]:
pd.set_option("display.max_columns",None)
df.sample(5)

,Title,Directed by,Story by,Based on,Produced by,Starring,Music by,Production company,Distributed by,Release dates,Running time,Country,Language,Budget,Box office,imdb_rating,imdb_votes,metascore,rotten_tomatoes_rating,Cinematography,Release date,Written by,Edited by,Screenplay by,Production companies
458,The Last Knight,Dmitriy Dyachenko,NaN,Baba Yaga and Koschei,"[Marina Zhigalova-Ozkan, Eduard Iloyan, Grigor...","[Viktor Khorinyak, Mila Sivatskaya, Konstantin...",George Kallis,NaN,Walt Disney Studios Sony Pictures Releasing CIS,NaN,114 minutes,Russia,NaN,"[~$8 million, ₽, 370 million]","[$30.5 million, ₽, 1.731 billion]",N/A,N/A,N/A,None,Sergei Trofimov,"[October 29, 2017 ( Russia )]",NaN,"[Tim Pavelko, Anton Anisimov, Aleksandr Amirov]","[Vitaliy Shlyappo, Dimitriy Yan, Pavel Danilov...","[Walt Disney Pictures CIS, Yellow, Black and W..."
356,Beverly Hills Chihuahua,Raja Gosnell,Jeff Bushell,NaN,"[David Hoberman, Todd Lieberman, John Jacobs, ...","[Piper Perabo, Manolo Cardona, Jamie Lee Curti...",Heitor Pereira,NaN,"[Walt Disney Studios, Motion Pictures]","[September 18, 2008 ( El Capitan Theatre ), Oc...",91 minutes,United States,NaN,$20 million,$149.3 million,3.9,"25,743",41,40%,Phil Méheux,NaN,NaN,Sabrina Plisco,"[Analisa LaBianco, Jeff Bushell]","[Walt Disney Pictures, Mandeville Films, Smart..."
259,The Emperor's New Groove,Mark Dindal,"[Chris Williams, Mark Dindal]","[Original story, by, Roger Allers, Matthew Jac...",Randy Fullmer,"[David Spade, John Goodman, Eartha Kitt, Patri...",John Debney,NaN,Buena Vista Pictures Distribution,"[December 10, 2000 ( El Capitan Theatre ), Dec...",78 minutes,United States,English,$100 million,$169.7 million,7.4,"223,097",70,86%,NaN,NaN,NaN,Pamela Ziegenhagen-Shefland,David Reynolds,"[Walt Disney Pictures, Walt Disney Feature Ani..."
383,The Sorcerer's Apprentice,Jon Turteltaub,"[Matt Lopez, Lawrence Konner, Mark Rosenthal]","[The Sorcerer's Apprentice, by, Johann Wolfgan...",Jerry Bruckheimer,"[Nicolas Cage, Jay Baruchel, Alfred Molina, Te...",Trevor Rabin,NaN,"[Walt Disney Studios, Motion Pictures]","[July 8, 2010 (Fantasia Film Festival), July 1...",109 minutes,United States,English,$150 million,$215.3 million,6.1,"169,303",46,40%,Bojan Bazelli,NaN,NaN,William Goldenberg,"[Doug Miro, Carlo Bernard, Matt Lopez]","[Walt Disney Pictures, Jerry Bruckheimer Films..."
316,Herbie: Fully Loaded,Angela Robinson,"[Thomas Lennon, Robert Ben Garant, Mark Perez]","[Characters, by Gordon Buford]",Robert Simonds,"[Lindsay Lohan, Justin Long, Breckin Meyer, Ma...",Mark Mothersbaugh,NaN,Buena Vista Pictures Distribution,"[May 10, 2005 ( Las Vegas Motor Speedway ), Ju...",100 minutes,United States,English,$50 million,$144.1 million,4.9,"50,466",47,41%,Greg Gardiner,NaN,NaN,Wendy Greene Bricmont,"[Thomas Lennon, Robert Ben Garant, Alfred Goug...","[Walt Disney Pictures, Robert Simonds Producti..."


### Running Type column changed to integer value

In [28]:
def running_time_num(x):
    if type(x) == list:
        x = x[0].split("(")[0]
        x = x.split(" ")[0]
        x = x.split("–")[0]
        return x
    elif pd.isnull(x):
        return np.nan
    else:
        x = x.split(" ")[0]
        x = x.split("–")[0]
        return x

In [29]:
df['Running time'] = df['Running time'].apply(running_time_num)

### Release dates column changed to datetime value

In [30]:
def datetime_format(x):
    if type(x) == list:
        x = x[0]
        x =x.split("(")[0]
        return x
    elif pd.isnull(x):
        return np.nan
    else:
        x =x.split("(")[0]
        return x

In [31]:
df['Release dates'] = pd.to_datetime(df['Release dates'].apply(datetime_format))

### Box Office and Budget column currency value converted to float value

In [32]:
import re

def string_start_with_dollor(value):
    try:
        num_fmt = "\$[0-9]*[.,]*[0-9]*[,]*[0-9]*"
        number = value[re.search(num_fmt,value).span()[0]+1:re.search(num_fmt,value).span()[1]]
        number = float(number.replace(",",""))
        return number
    except Exception as e:
        pass

def string_start_without_curr_sign(value):
    try:
        num_fmt = "[0-9]*[.,]*[0-9]*[,]*[0-9]*"
        number = re.search(num_fmt,value).group()
        number = float(number.replace(",",""))
        return number
    except Exception as e:
        pass

def string_start_with_rubel(value):
    try:
        num_fmt = "₽[0-9]*[.,]*[0-9]*[,]*[0-9]*"
        number = value[re.search(num_fmt,value).span()[0]+1:re.search(num_fmt,value).span()[1]]
        number = float(number.replace(",",""))*0.011
        return number
    except Exception as e:
        pass

def currency_numeric_converter(currency_str):
    try:
        if "$" in currency_str:
            number = string_start_with_dollor(currency_str)
            if "million" in currency_str:
                return number*1000000
            else:
                return number

        elif "₽" in currency_str:
            number = string_start_with_rubel(currency_str)
            if "million" in currency_str:
                return number*1000000
            else:
                return number
        else:
            number = string_start_without_curr_sign(currency_str)
            if "million" in currency_str:
                return number*1000000
            else:
                return number
    except Exception as e:
        pass
    
def money_converter(currency_str_df):
    try:
        if type(currency_str_df) == list:
            for item in currency_str_df:
                if "million" in item:
                    curr_value = currency_numeric_converter(item)
                    return curr_value

        elif pd.isnull(currency_str_df) or currency_str_df == "" or currency_str_df == ():
            return np.nan

        else:
            curr_value = currency_numeric_converter(currency_str_df)
            return curr_value
    except Exception as e:
        pass

In [33]:
df['Budget(in$)'] = df['Budget'].apply(money_converter)

In [34]:
df['Box office(in$)'] = df["Box office"].apply(money_converter)

In [35]:
df.drop(["Budget","Box office"],axis=1,inplace=True)

In [36]:
df.head()

,Title,Directed by,Story by,Based on,Produced by,Starring,Music by,Production company,Distributed by,Release dates,Running time,Country,Language,imdb_rating,imdb_votes,metascore,rotten_tomatoes_rating,Cinematography,Release date,Written by,Edited by,Screenplay by,Production companies,Budget(in$),Box office(in$)
0,Snow White and the Seven Dwarfs,"[David Hand, Perce Pearce, William Cottrell, L...","[Ted Sears, Richard Creedon, Otto Englander, D...","["", Snow White, "", by the, Brothers Grimm]",Walt Disney,"[Adriana Caselotti, Roy Atwell, Pinto Colvig, ...","[Frank Churchill, Leigh Harline, Paul Smith]",Walt Disney Productions,RKO Radio Pictures,1937-12-21,83,United States,English,7.6,"213,507",96,None,NaN,NaN,NaN,NaN,NaN,NaN,1500000.0,418000000.0
1,Pinocchio,"[Ben Sharpsteen, Hamilton Luske, Bill Roberts,...","[Ted Sears, Otto Englander, Webb Smith, Willia...","[The Adventures of Pinocchio, by, Carlo Collodi]",Walt Disney,"[Cliff Edwards, Dickie Jones, Christian Rub, W...","[Leigh Harline, Paul J. Smith]",Walt Disney Productions,RKO Radio Pictures,1940-02-07,88,United States,English,7.5,"158,057",99,100%,NaN,NaN,NaN,NaN,NaN,NaN,2600000.0,164000000.0
2,Fantasia,"[Samuel Armstrong, James Algar, Bill Roberts, ...","[Joe Grant, Dick Huemer]",NaN,"[Walt Disney, Ben Sharpsteen]","[Leopold Stokowski, Deems Taylor]",See plot,Walt Disney Productions,RKO Radio Pictures,NaT,126,United States,English,7.7,"103,427",96,95%,James Wong Howe,"[November 13, 1940]",NaN,NaN,NaN,NaN,2280000.0,76400000.0
3,The Reluctant Dragon,"[Alfred Werker, (live action), Hamilton Luske,...",NaN,NaN,Walt Disney,"[Robert Benchley, Frances Gifford, Buddy Peppe...","[Frank Churchill, Larry Morey]",Walt Disney Productions,RKO Radio Pictures,NaT,74,United States,English,6.7,"3,764",N/A,100%,Bert Glennon,"[June 27, 1941]","[Live-action:, Ted Sears, Al Perkins, Larry Cl...",Paul Weatherwax,NaN,NaN,600000.0,960000.0
4,Dumbo,"[Ben Sharpsteen, Norman Ferguson, Wilfred Jack...","[Joe Grant, Dick Huemer]","[Dumbo, the Flying Elephant, by, Helen Aberson...",Walt Disney,"[Edward Brophy, Verna Felton, Cliff Edwards, H...","[Frank Churchill, Oliver Wallace]",Walt Disney Productions,RKO Radio Pictures,1941-10-23,64,United States,English,7.2,"141,070",96,95%,NaN,NaN,NaN,NaN,NaN,NaN,950000.0,1300000.0


In [37]:
df.to_csv("Disney_movie_data.csv")